# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key = "178f2fc16c3a4230a1eb2b74ade88f58"

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\jdaiu\Python_API_Challenge\output_data\cities.csv", index_col="City_ID")



# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,namibe,-15.1961,12.1522,66.88,89,42,10.85,AO,1693417970
1,adamstown,-25.0660,-130.1015,70.16,90,99,20.40,PN,1693417970
2,petropavlovsk-kamchatsky,53.0452,158.6483,45.19,100,0,6.71,RU,1693417970
3,invercargill,-46.4000,168.3500,48.02,72,76,10.00,NZ,1693417970
4,talnakh,69.4865,88.3972,45.21,79,30,2.24,RU,1693417970


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[city_data_df["Max Temp"].between(65,80)]
ideal_weather = ideal_weather[ideal_weather["Wind Speed"] < 5]
ideal_weather = ideal_weather[ideal_weather["Cloudiness"] < 20]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna(how="any")

# Display sample data
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
36,saint-francois,46.4154,3.9054,67.44,56,0,4.61,FR,1693417975
55,dryden,49.7833,-92.7503,66.49,63,0,3.44,CA,1693417977
66,tura,25.5198,90.2201,76.71,79,3,2.68,IN,1693417978
152,khed,17.7167,73.3833,76.50,94,4,0.63,IN,1693417989
176,toliara,-23.3500,43.6667,70.93,78,1,0.31,MG,1693417992
205,uacu cungo,-11.3634,15.1208,74.62,18,2,2.17,AO,1693417997
233,duragan,41.4158,35.0544,78.66,46,0,4.27,TR,1693418001
243,port elizabeth,-33.9180,25.5701,72.10,52,0,0.00,ZA,1693418003
244,slave lake,55.2834,-114.7690,66.18,77,0,4.61,CA,1693417703


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
36,saint-francois,FR,46.4154,3.9054,56,
55,dryden,CA,49.7833,-92.7503,63,
66,tura,IN,25.5198,90.2201,79,
152,khed,IN,17.7167,73.3833,94,
176,toliara,MG,-23.3500,43.6667,78,
205,uacu cungo,AO,-11.3634,15.1208,18,
233,duragan,TR,41.4158,35.0544,46,
243,port elizabeth,ZA,-33.9180,25.5701,52,
244,slave lake,CA,55.2834,-114.7690,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-francois - nearest hotel: Chez Lily
dryden - nearest hotel: Holiday Inn Express
tura - nearest hotel: No hotel found
khed - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
uacu cungo - nearest hotel: Ritz Waku Kungu
duragan - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
slave lake - nearest hotel: Northwest Inn
sinop - nearest hotel: No hotel found
yunfu - nearest hotel: 九鼎明珠酒店
santo amaro da imperatriz - nearest hotel: Águas Mornas Palace Hotel
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
kruisfontein - nearest hotel: No hotel found
son la - nearest hotel: Khách sạn Mường Thanh Luxury Sơn La
bhainsdehi - nearest hotel: No hotel found
sarupathar - nearest hotel: No hotel found
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
jiangyou - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
36,saint-francois,FR,46.4154,3.9054,56,Chez Lily
55,dryden,CA,49.7833,-92.7503,63,Holiday Inn Express
66,tura,IN,25.5198,90.2201,79,No hotel found
152,khed,IN,17.7167,73.3833,94,No hotel found
176,toliara,MG,-23.3500,43.6667,78,Ambary
205,uacu cungo,AO,-11.3634,15.1208,18,Ritz Waku Kungu
233,duragan,TR,41.4158,35.0544,46,No hotel found
243,port elizabeth,ZA,-33.9180,25.5701,52,Waterford Hotel
244,slave lake,CA,55.2834,-114.7690,77,Northwest Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)